In [1]:
import os
from pathlib import Path

p = Path.cwd()
os.chdir(p.parent)
os.getcwd()

'/root/repos/lightning/HousePricing'

In [2]:
from dotenv import dotenv_values
import os

envs = ["secret.env", "predict.env"]

for fenv in envs:
    file = os.path.join("env", fenv)
    config = dotenv_values(file)  # load sensitive variables
    print(config.keys())
    for c, v in config.items():
        os.environ[c] = v

odict_keys(['WANDB_API_KEY', 'WANDB_NAME'])
odict_keys(['WANDB_NOTEBOOK_NAME'])


In [3]:
import pandas as pd

artifacts = pd.read_csv("artifacts.csv")
artifacts

,run_name,best_model,fit_accelerator,fit_max_epochs,fit_patience,fit_lr,fit_batch_size,fit_in_features,fit_validation_size
0,Mr._ZI1EUZUC,./lightning_logs/mrsat7bm/checkpoints/house_pr...,gpu,1000,100,0.001,256,244,0.1
1,Gerald_4LIP1BLK,./lightning_logs/3478itnz/checkpoints/house_pr...,gpu,1000,100,0.001,256,244,0.1
2,Stacey_Y41HGQZW,./lightning_logs/27yrkxwa/checkpoints/house_pr...,gpu,1000,100,0.001,256,244,0.1


In [4]:
artifacts = artifacts.to_dict(orient="records")
artifacts

[{'run_name': 'Mr._ZI1EUZUC',
  'best_model': './lightning_logs/mrsat7bm/checkpoints/house_pricing-epoch=956-val_loss=383568800.00.ckpt',
  'fit_accelerator': 'gpu',
  'fit_max_epochs': 1000,
  'fit_patience': 100,
  'fit_lr': 0.001,
  'fit_batch_size': 256,
  'fit_in_features': 244,
  'fit_validation_size': 0.1},
 {'run_name': 'Gerald_4LIP1BLK',
  'best_model': './lightning_logs/3478itnz/checkpoints/house_pricing-epoch=831-val_loss=389454272.00.ckpt',
  'fit_accelerator': 'gpu',
  'fit_max_epochs': 1000,
  'fit_patience': 100,
  'fit_lr': 0.001,
  'fit_batch_size': 256,
  'fit_in_features': 244,
  'fit_validation_size': 0.1},
 {'run_name': 'Stacey_Y41HGQZW',
  'best_model': './lightning_logs/27yrkxwa/checkpoints/house_pricing-epoch=818-val_loss=392531328.00.ckpt',
  'fit_accelerator': 'gpu',
  'fit_max_epochs': 1000,
  'fit_patience': 100,
  'fit_lr': 0.001,
  'fit_batch_size': 256,
  'fit_in_features': 244,
  'fit_validation_size': 0.1}]

In [5]:
from src.data import HousePricingDataModule

data_module = HousePricingDataModule()
data_module.prepare_data()

[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...


In [6]:
import os
from lightning import Trainer
import torch

from pytorch_lightning.loggers import WandbLogger

from src.model import NeuralNetwork

project_name = os.environ["WANDB_NAME"]

accelerator = "gpu" if torch.cuda.is_available() else "cpu"
in_features = data_module.data_features()

config = {
    "accelerator": accelerator,
    "in_features": in_features,
}

predictions = dict()

for artifact in artifacts:

    run_name = artifact["run_name"]
    best_model = artifact["best_model"]

    model = NeuralNetwork.load_from_checkpoint(
        checkpoint_path=best_model, input_size=in_features
    )

    trainer = Trainer(accelerator=config["accelerator"])

    prediction = trainer.predict(model, datamodule=data_module)

    predictions[run_name] = prediction

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[INFO]: Input size: 244


/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...
[INFO]: Setting up predict dataset/s
[INFO]: Predict dataloader size: 12
Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 72.33it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

[INFO]: Input size: 244
[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[INFO]: Setting up predict dataset/s
[INFO]: Predict dataloader size: 12


/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 62.93it/s]


In [7]:
def prediction_to_submit(prediction_batches):

    submit = {"Id": [], "SalePrice": []}

    for prediction_batch in prediction_batches:
        ids = prediction_batch["id"].tolist()
        predictions = prediction_batch["prediction"].tolist()

        submit["Id"] = submit["Id"] + ids
        submit["SalePrice"] = submit["SalePrice"] + predictions

    return submit

In [8]:
predictions

{'Mr._ZI1EUZUC': [{'id': tensor([1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470, 1471, 1472,
           1473, 1474, 1475, 1476, 1477, 1478, 1479, 1480, 1481, 1482, 1483, 1484,
           1485, 1486, 1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496,
           1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508,
           1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520,
           1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532,
           1533, 1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544,
           1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556,
           1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 1568,
           1569, 1570, 1571, 1572, 1573, 1574, 1575, 1576, 1577, 1578, 1579, 1580,
           1581, 1582, 1583, 1584, 1585, 1586, 1587, 1588], dtype=torch.int32),
   'prediction': tensor([132965.7344, 165273.5156, 185591.7656, 2006

In [10]:
for run_name, prediction in predictions.items():
    submit = prediction_to_submit(prediction)
    submit_csv = pd.DataFrame(submit)
    submit_csv.to_csv(f"submit-{run_name}.csv", index=False)